## Parte I - Leitura dos Dados

In [ ]:
!pip install PyDrive &> /dev/null
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#Para baixar o banco de dados original
downloaded = drive.CreateFile({'id':"1A3Lz3ZPRlIEqEab3x3skWjFOJGyai1ZN"})
downloaded.GetContentFile('PredicaoAtributosVisuais.zip')

In [ ]:
!unzip -q PredicaoAtributosVisuais.zip #Descompreção do arquivo de dados 

## Código

In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
import keras
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
from keras import layers
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras import models
from IPython.display import Image, display
import matplotlib.cm as cm
%matplotlib inline

model = VGG16(weights='imagenet')

def returnAnnotations(file_attributos, file_annotations):
  df_attributes = pd.read_csv(file_attributos, header = None)
  list_attributes = list()
  list_attributes.append(df_attributes[0].values)
  handle = open(file_annotations)
  scores = dict()
  confidence = dict()
  for line in handle:
      words = line.split()
      i=0
      for word in words:
          if i == 0:
              scores[word]=list()
              confidence[word] = list()
              i+=1
          else:
              values = word.split(',')
              scores[words[0]].append(float(values[0]))
              confidence[words[0]].append(float(values[1]))
              i+=1
  scores = pd.DataFrame(scores).T
  scores.columns = list_attributes
  confidence = pd.DataFrame(confidence).T
  confidence.columns = list_attributes
  return scores,confidence

file_attributos = './Predição de Atributos Visuais/annotations/attributes.txt'
file_annotations = './Predição de Atributos Visuais/annotations/annotations.tsv'
file_training = './Predição de Atributos Visuais/holdout_split/training.txt'
file_test = './Predição de Atributos Visuais/holdout_split/test.txt'
def returnTrainingTest(file_attributos, file_annotations, file_training, file_test):
  scores,confidence = returnAnnotations(file_attributos, file_annotations)
  df_training = pd.read_csv(file_training, header = None)
  df_test = pd.read_csv(file_test, header = None)
  scores_training = scores[scores.index.isin(df_training[0].values)]
  scores_test = scores[scores.index.isin(df_test[0].values)]
  confidence_training = confidence[confidence.index.isin(df_training[0].values)]
  confidence_test = confidence[confidence.index.isin(df_test[0].values)]
  return scores_training, scores_test, confidence_training, confidence_test

scores_training, scores_test, confidence_training, confidence_test = returnTrainingTest(file_attributos, file_annotations, file_training, file_test)

df_training = scores_training
df_test = scores_test

import os
path_drive = os.getcwd()
def returnPathImages(path):
  imageLD = dict()
  imageAlignedLD = dict()
  pastas = ['imageLD', 'imageAlignedLD']
  path_imagens=list()
  for pasta in pastas:
    dataset = os.listdir(path+'/Predição de Atributos Visuais/'+pasta+'/')
    for data in dataset:
      if data != '.DS_Store' and pasta=='imageLD':
        imagenes = os.listdir(path+'/Predição de Atributos Visuais/'+pasta+'/'+data+'/')
        for imagen in imagenes:
          path_imagen = path+'/Predição de Atributos Visuais/'+pasta+'/'+data+'/'+imagen
          imageLD[data+'/'+imagen]=path_imagen
      if data != '.DS_Store' and pasta=='imageAlignedLD':
        imagenes = os.listdir(path+'/Predição de Atributos Visuais/'+pasta+'/'+data+'/')
        for imagen in imagenes:
          path_imagen = path+'/Predição de Atributos Visuais/'+pasta+'/'+data+'/'+imagen
          imageAlignedLD[data+'/'+imagen]=path_imagen
  imageLD = pd.DataFrame(data=imageLD.values(), index=imageLD.keys(), columns=['Path'])
  imageAlignedLD = pd.DataFrame(data=imageAlignedLD.values(), index=imageAlignedLD.keys(), columns=['Path'])
  return imageLD, imageAlignedLD

imageLD, imageAlignedLD = returnPathImages(path_drive)
image_path = pd.concat([imageLD, imageAlignedLD], axis=0)
df_training_imageAlignedLD = df_training[df_training.index.isin(imageAlignedLD.index)]

df_training_imageLD = df_training[df_training.index.isin(imageLD.index)]
df_test_imageAlignedLD = df_test[df_test.index.isin(imageAlignedLD.index)]

import numpy as np

def returnImageByIndex(i, df_path, df_training):
  img = cv2.imread(df_path[df_path.index==df_training.index[i]].values[0][0])
  return img

Path_imageLD = imageLD[imageLD.index.isin(df_training_imageLD.index)]
Path_imageLD = pd.concat([Path_imageLD, df_training_imageLD], axis=1)
Path_imageAlignedLD = imageAlignedLD[imageAlignedLD.index.isin(df_training_imageAlignedLD.index)]
Path_imageAlignedLD = pd.concat([Path_imageAlignedLD, df_training_imageAlignedLD], axis=1)
data_treino = pd.concat([Path_imageLD, Path_imageAlignedLD], axis=0)

def visualizeImageByIndex(i, df_path, df_training):
  img = cv2.imread(df_path[df_path.index==df_training.index[i]].values[0][0])
  cv2_imshow(img)
  del img

def returnImageByIndex(i, df_path, df_training):
  img = cv2.imread(df_path[df_path.index==df_training.index[i]].values[0][0])
  return img

def int_layer(indice_img):
    img_pre = returnImageByIndex(indice_img, df_path= imageLD, df_training = df_training_imageLD)
    img = tf.image.resize(img_pre, [224,224])
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_tensor = img_arr/255.
    visualizeImageByIndex(indice_img, df_path= imageLD, df_training = df_training_imageLD)

    # Extracts the outputs of the top 8 layers:
    layer_outputs = [layer.output for layer in model.layers[1:]]
    # Creates a model that will return these outputs, given the model input:
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(img_tensor)
    first_layer_activation = activations[0]

    # These are the names of the layers, so can have them as part of our plot
    layer_names = []
    for layer_id in range(1, len(model.layers[:-1]), 6):
        layer_names.append(model.layers[layer_id].name)

    images_per_row = 12

    # Now let's display our feature maps
    for layer_name, layer_activation in zip(layer_names, activations):
        # This is the number of features in the feature map
        n_features = layer_activation.shape[-1]

        # The feature map has shape (1, size, size, n_features)
        size = layer_activation.shape[1]

        # We will tile the activation channels in this matrix
        n_cols = n_features // images_per_row
        display_grid = np.zeros((size * n_cols, images_per_row * size))

        # We'll tile each filter into this big horizontal grid
        for col in range(n_cols):
            for row in range(images_per_row):
                channel_image = layer_activation[0,
                                                :, :,
                                                col * images_per_row + row]
                # Post-process the feature 
                channel_image -= channel_image.mean()
                channel_image /= channel_image.std()
                channel_image *= 64
                channel_image += 128
                channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                display_grid[col * size : (col + 1) * size,
                            row * size : (row + 1) * size] = channel_image

        # Display the grid
        scale = 1. / size
        plt.figure(figsize=(3 * scale * display_grid.shape[1],
                        3 * scale * display_grid.shape[0]))
        plt.title(layer_name)
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='cividis')
    
    plt.show()

def first_layer(indice_img):
    img_pre = returnImageByIndex(indice_img, df_path= imageLD, df_training = df_training_imageLD)
    img = tf.image.resize(img_pre, [224,224])
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_tensor = img_arr/255.
    
    # Extracts the outputs of the top 8 layers:
    layer_outputs = [layer.output for layer in model.layers[1:]]
    # Creates a model that will return these outputs, given the model input:
    activation_model = models.Model(inputs=model.input, outputs=layer_outputs)
    activations = activation_model.predict(img_tensor)
    first_layer_activation = activations[0]

    plt.matshow(first_layer_activation[0, :, :, 1], cmap='cividis')
    plt.axis('off')
    plt.show()

def h_map(indice_img):

    img_path1 = imageLD[imageLD.index==df_training_imageLD.index[indice_img]].values[0][0]
    img_pre = returnImageByIndex(indice_img, df_path= imageLD, df_training = df_training_imageLD)
    img = tf.image.resize(img_pre, [224,224])
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_tensor = img_arr/255.

    def get_img_array(img_path, size):
        # `img` is a PIL image of size 299x299
        img = keras.preprocessing.image.load_img(img_path, target_size=size)
        # `array` is a float32 Numpy array of shape (299, 299, 3)
        array = keras.preprocessing.image.img_to_array(img)
        # We add a dimension to transform our array into a "batch"
        # of size (1, 299, 299, 3)
        array = np.expand_dims(array, axis=0)
        return array


    def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
        # First, we create a model that maps the input image to the activations
        # of the last conv layer as well as the output predictions
        grad_model = tf.keras.models.Model(
            [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
        )

        # Then, we compute the gradient of the top predicted class for our input image
        # with respect to the activations of the last conv layer
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]

        # This is the gradient of the output neuron (top predicted or chosen)
        # with regard to the output feature map of the last conv layer
        grads = tape.gradient(class_channel, last_conv_layer_output)

        # This is a vector where each entry is the mean intensity of the gradient
        # over a specific feature map channel
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

        # We multiply each channel in the feature map array
        # by "how important this channel is" with regard to the top predicted class
        # then sum all the channels to obtain the heatmap class activation
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)

        # For visualization purpose, we will also normalize the heatmap between 0 & 1
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
        return heatmap.numpy()
  
    # Remove last layer's softmax
    model.layers[-1].activation = None
    last_conv_layer_name = "block5_conv3"

    # Generate class activation heatmap
    heatmap0 = make_gradcam_heatmap(img_tensor, model, last_conv_layer_name)

    import matplotlib.cm as cm
    from IPython.display import Image, display

    def save_and_display_gradcam(img_path, heatmap, cam_path="cam.jpg", alpha=0.4):
        # Load the original image
        img = keras.preprocessing.image.load_img(img_path)
        img = keras.preprocessing.image.img_to_array(img)

        # Rescale heatmap to a range 0-255
        heatmap = np.uint8(255 * heatmap)

        # Use jet colormap to colorize heatmap
        jet = cm.get_cmap("jet")

        # Use RGB values of the colormap
        jet_colors = jet(np.arange(256))[:, :3]
        jet_heatmap = jet_colors[heatmap]

        # Create an image with RGB colorized heatmap
        jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
        jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
        jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

        # Superimpose the heatmap on original image
        superimposed_img = jet_heatmap * alpha + img
        superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

        # Save the superimposed image
        superimposed_img.save(cam_path)

        # Display Grad CAM
        display(Image(cam_path))
    
    display(Image(img_path1))
    save_and_display_gradcam(img_path1, heatmap0)

## Parte II - Visualização Das Camadas Intermediárias

In [ ]:
indice_imagem = 447 # Indice da imagem a ser visualizada
int_layer(indice_imagem)

In [ ]:
indice_imagem = 2589 # Indice da imagem a ser visualizada
int_layer(indice_imagem)

In [ ]:
indice_imagem = 777 # Indice da imagem a ser visualizada
int_layer(indice_imagem)

In [ ]:
first_layer(indice_imagem)

Podemos notar que no cenário, em dia de neve, além dos contornos existe uma tonalidade de cores mais clara sendo enviada à rede neural.

In [ ]:
first_layer(2589)

O mesmo cenário, num dia ensolarado, mostra uma tonalidade de cores mais escura sendo enviada à rede neural.



```
# Isto está formatado como código
```

## Parte III - Mapas de Calor

In [ ]:
indice_imagem = 3432 # Indice da imagem a ser visualizada
h_map(indice_imagem)

In [ ]:
indice_imagem = 1444 # Indice da imagem a ser visualizada
h_map(indice_imagem)

In [ ]:
indice_imagem = 2233 # Indice da imagem a ser visualizada
h_map(indice_imagem)

In [ ]:
indice_imagem = 3333 # Indice da imagem a ser visualizada
h_map(indice_imagem)

In [ ]:
indice_imagem = 1117 # Indice da imagem a ser visualizada
h_map(indice_imagem)

In [ ]:
indice_imagem = 1333 # Indice da imagem a ser visualizada
h_map(indice_imagem)

In [ ]:
indice_imagem = 1717 # Indice da imagem a ser visualizada
h_map(indice_imagem)

Fica evidente que dependendo do clima/estação do ano, a rede neural busca pontos bem diferentes da imagem para classificá-la.